In [258]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from numpy.linalg import inv
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import cho_solve, cho_factor

%matplotlib inline

In [748]:
def sinc(x):
    return np.sin(np.pi * x) / (np.pi * x)

def objective(x):
    return sinc(x+10) + sinc(x) + sinc(x-10)

pts = 250
X = np.linspace(-15,15,pts) # value locations
y = objective(X) # objective function

D = np.concatenate([X[X<=-4.5], X[X>=4.5]]) # value locations we know
unk = X[np.logical_and(X>-4.5, X<4.5)]


# def dist(X1, X2):
#     G1 = (X1**2).reshape(-1,1)
#     G2 = (X2**2).reshape(-1,1)
#     return np.sqrt(G1 + G2.T - 2 * np.dot(X1,X2.T))

def dist(X, X2):
        #Distance
        if X2 is None:
            X2 = X 
        X = np.expand_dims(X.T, axis=2)
        X2 = np.expand_dims(X2.T, axis=1)                                                                          
        return np.matmul(X, np.ones_like(X2)) + np.matmul(np.ones_like(X), -X2)


    
def kernSM(X1, X2, weights, means, covs):
    n1 = X1.shape[1]
    n2 = X2.shape[1]
    X1r = np.repeat(X1, n2, axis=0)
    X2r = np.repeat(X2.T, n1, axis=1)
    d = X1r - X2r
    Q = weights.shape[0]
    K = 0
    for i in range(Q):
        w = weights[i]
        m = means[i]
        c = covs[i]
        Ki = w * np.exp(-2 * np.pi ** 2 * c * np.square(d)) * np.cos(2 * np.pi * d * m)
        K += Ki
#     print(f'{np.linalg.det(K)} {np.linalg.det(K.T)}')
    return K.T
        
    
def predict(X, X_train, Y_train, kernFunc, weights, means, covs, train=False):
    C = kernFunc(X_train, X_train, weights, means, covs) + .001 * np.eye(X_train.shape[0])
    kappa = kernFunc(X_train, X, weights, means, covs)
    K = kernFunc(X, X, weights, means, covs) +  .001 * np.eye(X.shape[0])
    C_inv = inv(C)
#     if train:
#         try:
#             cho = cho_factor(C,lower=True)
#             L = cho[0]
#             lower = cho[1]
#             upper = not lower
#             v = cho_solve((L.T, upper), kappa)
#             b = cho_solve(cho, Y_train)
#             alpha = cho_solve((L.T, upper), b)

#             u = kappa.T.dot(alpha)
#             cov = K - np.dot(v.T, v)
#             ml = -.5 * (Y_train.T.dot(alpha) + 2 * np.sum(np.log(np.diag(L))) + len(X_train) * np.log(2 * np.pi))
#             return u, cov, ml
#         except:
#             return np.zeros_like(X), K, np.array([[-1000]])
    u = kappa.T.dot(C_inv).dot(Y_train)
    cov = K - kappa.T.dot(C_inv).dot(kappa)
    return u, cov
    
        
# hyps = [weight vector, mean vector, covariance vector]
def marginalLikelihood(hyps, objval, xInds, Q, sig, train):
    hyps = hyps.flatten()
        
    w = hyps[:Q].reshape(-1,1)
    m = hyps[Q:2*Q].reshape(-1,1)
    cov = hyps[2*Q:].reshape(-1,1)
    
    condMean, condKern = predict(xInds, xInds, objective(xInds), kernSM, w, m, cov, train)
#     print(f'{np.linalg.det(condKern)} {ml[0,0]}')
#     return -ml[0,0]
    ml = - .5 * (objval - condMean).T @ inv(condKern + sig) @ (objval - condMean) - .5 * np.linalg.det(condKern + sig) - len(xInds)/2 * np.log(2*np.pi)
#     print(f'{np.linalg.det(condKern)} {ml[0,0]}')
    return ml[0,0]

In [749]:
Q = 7

Dvect = D.reshape(-1,1)
hyps = np.random.random(3 * Q) * 1e-8
noise = .001 * np.eye(Dvect.shape[0])
knownData = objective(Dvect)

# hyps = np.concatenate([Wvect, Mvect, Cvect])

In [750]:
optHyps = optimize.fmin_cg(marginalLikelihood, hyps, args=(knownData, Dvect, Q, noise, True), epsilon=1e-6, gtol=-1e20)

C:\Users\Patrick\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in add


Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 2576
         Gradient evaluations: 112


C:\Users\Patrick\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2093: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\Patrick\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2093: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [747]:
u_posterior, K_posterior = predict(unk.reshape(-1,1), Dvect, knownData, kernSM, optw, optm, optcov)
post_samples = np.random.multivariate_normal(u_posterior.ravel(), K_posterior, 1)
plt.plot(unk, u_posterior)
plt.plot(X, y)

ValueError: array must not contain infs or NaNs